# 📊 Parcial – Análisis de Redes Sociales (Primer Corte)

## 🎯 Objetivo
Evaluar tu dominio de **filtros** y **agrupaciones** en Python (sin pivots ni gráficos) y tu capacidad de **análisis de negocio** en métricas de marketing: **CAC, LTV, churn rate, funnel, unit economics (LTV/CAC)**.

---
## 🧩 Contexto de Negocio (Caso Real)
**StartUp SaaS en crecimiento.** Vende planes **Basic, Pro, Enterprise** en **LatAm, North America, Europe, APAC**. Los canales de adquisición son: **meta_ads, google_ads, organic_search, outbound_sales**. Los usuarios se registran por **web** o **mobile_app**.

La dirección quiere decidir **dónde invertir el presupuesto del próximo trimestre**. Te piden:

1) Identificar **canales más rentables** (relación **LTV/CAC** y churn).  
2) Detectar **regiones** con mayor **potencial de crecimiento** y/o **riesgo**.  
3) Evaluar el desempeño por **tier** (Basic, Pro, Enterprise).  
4) Recomendar **asignación de presupuesto** por **canal + tier**.

---
## 📂 Instrucciones
- Trabaja únicamente con **filtros** y **agrupaciones**. **No uses** tablas dinámicas (pivot), merges ni gráficos.
- Archivo a usar: `clientes_marketing.csv`.
- Escribe **código + interpretación breve** para cada inciso.
- Si el enunciado dice **“Agrupa”**, usa `groupby`. Si dice **“Filtra”**, usa filtrado con máscaras booleanas. Si dice **“Ambas”**, realiza primero el **filtro** y luego la **agrupación**.

---


## Descargar Tabla (Correr una vez)

In [ ]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv

--2025-09-18 03:26:09--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv [following]
--2025-09-18 03:26:09--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661398 (646K) [text/plain]
Saving to: ‘cac_ltv_model.csv’

cac_ltv_model.csv   100%[===================>] 645.90K  --.-KB/s

## Importar Datos

In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# 1) Cargar dataset
data = pd.read_csv('cac_ltv_model.csv')


### ✅ Variables relevantes (recordatorio)
- `acquisition_channel`, `signup_source`, `region`, `customer_tier`
- `plan_price`, `discount_rate`, `arpu`, `gross_margin`, `churn_rate`
- `contract_length_months`, `marketing_spend`

### 🧮 Fórmulas de negocio (a usar en el examen)
- **CAC por canal**:  
    $$ CAC = \frac{\text{Total Marketing Spend}}{\text{Clientes adquiridos}} $$
  
- **LTV por observación** (aprox.):  
   $$ LTV = ARPU \times contract\_length\_months \times gross\_margin \times (1 - churn\_rate) $$

- **Unit economics**:  
  $$ LTV/CAC   $$(ideal \> 3 como regla general de SaaS)

---



### Parte Exploratoria: Cree una muestra de los primeros 5 elementos de esta tabla. ¿Cuantas observaciones tienes?

In [ ]:
display(data.head())
print(f"Número de observaciones: {data.shape[0]}")

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27


Número de observaciones: 7057


## 🔎 Parte A – Filtros

hint: Crea una tabla con el filtro y luego agrupa la variable que te piden.

1. **Filtra** los clientes del canal **`meta_ads`**. Calcula el **ARPU promedio** de este subconjunto.  



El ARPU promedio para clientes de meta_ads es: 168.79

In [ ]:
data_meta_ads = data[data['acquisition_channel'] == 'meta_ads']
arpu_meta_ads = data_meta_ads['arpu'].mean()

print(f"El ARPU promedio para clientes de meta_ads es: {arpu_meta_ads:,.2f}")

2. **Filtra** clientes de **LatAm** con **`churn_rate > 0.05`**. ¿Cuántos son? ¿Cuál es su **ARPU promedio**?  

Número de clientes de LatAm con churn_rate > 0.05  410 ARPU promedio de estos clientes: 174.78

In [ ]:
data_latam = data[(data['region'] == 'LatAm') & (data['churn_rate'] > 0.05)]
num_clientes_latam = len(data_latam)
arpu_promedio_latam = data_latam['arpu'].mean()

print(f"Número de clientes de LatAm con churn_rate > 0.05: {num_clientes_latam}")
print(f"ARPU promedio de estos clientes: {arpu_promedio_latam:,.2f}")

3. **Filtra** clientes **Enterprise** con `contract_length_months > 6`. ¿Cuál es su **gross_margin promedio**?

El gross_margin promedio para clientes Enterprise con contrato > 6 meses es: 0.82

In [ ]:
data_enterprise = data[(data['customer_tier'] == 'Enterprise') & (data['contract_length_months'] > 6)]
gross_margin_enterprise = data_enterprise['gross_margin'].mean()

print(f"El gross_margin promedio para clientes Enterprise con contrato > 6 meses es: {gross_margin_enterprise:,.2f}")

## 📊 Parte B – Agrupaciones (SOLO agrupar)
4. **Agrupa** por `acquisition_channel` y calcula **ARPU promedio**. Ordena de mayor a menor.  


In [ ]:
arpu_por_canal = data.groupby('acquisition_channel')['arpu'].mean().sort_values(ascending=False)
display(arpu_por_canal)

,arpu
acquisition_channel,
outbound_sales,173.61
google_ads,172.37
organic_search,171.14
meta_ads,168.79


5. **Agrupa** por `region` y calcula **churn_rate promedio**. Identifica la región con mayor churn.  



Las regiones son similares

In [ ]:
churn_rate_por_region = data.groupby('region')['churn_rate'].mean().sort_values(ascending=False)
display(churn_rate_por_region)
region_mayor_churn = churn_rate_por_region.index[0]

,churn_rate
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05


6. **Agrupa** por `customer_tier` y calcula **marketing_spend total**. ¿Cuál tier consume más presupuesto?

El tier que consume más presupuesto de marketing es: Pro

In [ ]:
marketing_spend_por_tier = data.groupby('customer_tier')['marketing_spend'].sum().sort_values(ascending=False)
display(marketing_spend_por_tier)
tier_mayor_gasto = marketing_spend_por_tier.index[0]

,marketing_spend
customer_tier,
Pro,"589,265.04"
Basic,"584,419.11"
Enterprise,"584,348.87"


## 🔀 Parte C – Filtro **y** Agrupación (Ambas)
7. **Filtra** solo registros de **`signup_source = 'web'`** y luego **agrupa** por `acquisition_channel` para obtener el **churn_rate promedio**.  


In [ ]:
data_web = data[data['signup_source'] == 'web']
churn_rate_web_por_canal = data_web.groupby('acquisition_channel')['churn_rate'].mean()
display(churn_rate_web_por_canal)

,churn_rate
acquisition_channel,
google_ads,0.05
meta_ads,0.05
organic_search,0.05
outbound_sales,0.05


8. **Filtra** solo **`mobile_app`** y luego **agrupa** por `region` para calcular **ARPU promedio**.  


In [ ]:
data_mobile_app = data[data['signup_source'] == 'mobile_app']
arpu_mobile_app_por_region = data_mobile_app.groupby('region')['arpu'].mean()
display(arpu_mobile_app_por_region)

,arpu
region,
APAC,169.09
Africa,172.60
Europe,165.89
LatAm,173.20
Middle East,171.71
North America,174.00


9. **Filtra** a clientes **`Pro`** y **agrupa** por `acquisition_channel` para calcular **marketing_spend total**.


In [ ]:
data_pro = data[data['customer_tier'] == 'Pro']
marketing_spend_pro_por_canal = data_pro.groupby('acquisition_channel')['marketing_spend'].sum()
display(marketing_spend_pro_por_canal)

,marketing_spend
acquisition_channel,
google_ads,"226,721.68"
meta_ads,"211,517.60"
organic_search,"29,848.72"
outbound_sales,"121,177.04"


## 📈 Parte D – Métricas de negocio (CAC, LTV, LTV/CAC)
10. **CAC por canal (Agrupa)**: calcula el CAC de cada `acquisition_channel` como:  
   `CAC = marketing_spend_total_del_canal / #clientes_del_canal`  


In [ ]:
cac_por_canal = data.groupby('acquisition_channel').agg(
    total_marketing_spend=('marketing_spend', 'sum'),
    num_clientes=('customer_id', 'count')
)
cac_por_canal['CAC'] = cac_por_canal['total_marketing_spend'] / cac_por_canal['num_clientes']
display(cac_por_canal[['CAC']])

,CAC
acquisition_channel,
google_ads,401.02
meta_ads,349.27
organic_search,49.95
outbound_sales,200.22


11. **CAC por tier (Agrupa)**: calcula el CAC de cada `customer_tier` como:  
   `CAC = marketing_spend_total_del_tier / #clientes_del_canal`  


In [ ]:
cac_por_tier = data.groupby('customer_tier').agg(
    total_marketing_spend=('marketing_spend', 'sum'),
    num_clientes=('customer_id', 'count')
)
cac_por_tier['CAC'] = cac_por_tier['total_marketing_spend'] / cac_por_tier['num_clientes']
display(cac_por_tier[['CAC']])

,CAC
customer_tier,
Basic,249.75
Enterprise,249.19
Pro,248.43


12. **LTV por tier (Ambas)**: crea una columna `ltv_individual` con la fórmula dada.


In [ ]:
data['ltv_individual'] = data['arpu'] * data['contract_length_months'] * data['gross_margin'] * (1 - data['churn_rate'])
display(data.head()) # Display the first few rows to show the new column

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend,ltv_individual
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48,567.54
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60,99.20
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40,"2,651.91"
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10,139.68
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27,328.72


12. A. Luego **agrupa** `ltv_individual` por
`acquisition_channel` para obtener el **LTV promedio**.  

In [ ]:
ltv_por_canal = data.groupby('acquisition_channel')['ltv_individual'].mean()
display(ltv_por_canal)

,ltv_individual
acquisition_channel,
google_ads,710.25
meta_ads,729.05
organic_search,765.13
outbound_sales,751.57


12. B. Luego **agrupa** `ltv_individual`por `customer_tier` para obtener el **LTV promedio**.





In [ ]:
ltv_por_tier = data.groupby('customer_tier')['ltv_individual'].mean()
display(ltv_por_tier)

,ltv_individual
customer_tier,
Basic,291.77
Enterprise,"1,349.02"
Pro,577.63


13. **Unit economics (Ambas)**: combina tus resultados para comparar **LTV promedio por tier** contra **CAC por canal** y comenta **qué combinaciones canal + tier** lucen más saludables (busca **LTV/CAC > 3**).


In [ ]:
ltv_por_canal_tier = data.groupby(['acquisition_channel', 'customer_tier'])['ltv_individual'].mean().unstack()
display(ltv_por_canal_tier)

customer_tier,Basic,Enterprise,Pro
acquisition_channel,,,
google_ads,268.95,"1,271.47",560.24
meta_ads,314.96,"1,353.77",558.70
organic_search,288.84,"1,410.71",596.45
outbound_sales,293.15,"1,361.48",594.23


In [ ]:
cac_por_canal_tier = data.groupby(['acquisition_channel', 'customer_tier']).agg(
    total_marketing_spend=('marketing_spend', 'sum'),
    num_clientes=('customer_id', 'count')
)
cac_por_canal_tier['CAC'] = cac_por_canal_tier['total_marketing_spend'] / cac_por_canal_tier['num_clientes']
display(cac_por_canal_tier[['CAC']])

CAC
acquisition_channel customer_tier       
google_ads          Basic         399.95
                    Enterprise    401.79
                    Pro           401.28
meta_ads            Basic         350.81
                    Enterprise    347.26
                    Pro           349.62
organic_search      Basic          49.81
                    Enterprise     50.06
                    Pro            50.00
outbound_sales      Basic         200.40
                    Enterprise    199.98
                    Pro           200.29

## 🧠 Parte E – Análisis crítico (respuesta abierta)
13. **CMO por un día**: con tus resultados, ¿en qué **canal(es)** invertirías más el próximo trimestre? ¿Por qué? (Cita **CAC** y **churn**).  
14. ¿Qué **región** ves con mayor riesgo? ¿Qué hipótesis explicarían su **churn**?  
15. Identifica **dos combinaciones canal + tier** con mejor **LTV/CAC** y propón **dos acciones** concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).

---
✍️ **Nota**: Mantente disciplinado con el enunciado (**Filtra**, **Agrupa**, **Ambas**). La evaluación pondera **correctitud técnica** y **calidad de interpretación de negocio**.


13. Invertir más en canales con alta LTV/CAC y churn bajo o razonable. En particular, si el análisis muestra que:
Organic Search tiene CAC bajo y LTV por observación alto, oLTV/CAC > 3 en varias tiers. Un CAC bajo combinado con LTV alto mejora la métrica LTV/CAC y churn bajo ayuda a sostener LTV a largo plazo.

14. La región con churn_rate más alto es la que, en tu dataset, presenta el valor promedio más alto (la que resulta de la agrupación por región). Hipotesis como Soporte/onboarding insuficiente, Competencia regional fuerte,Desalineación de producto con necesidades locales

15.Organic Search + Enterprise creando contenido tecnico y fortaleciendo inbound: webinars,onbording y personalizado.
Google Ads + Pro debido a que pueden hacer segmentación precisa y pruebas A/B de mensajes y landing pages enfocadas en problemas que Pro resuelve y pruebas de paquetes y ofertas temporales